# ViT : Vision Transformer

### ViT Overview
ViT는 Transformer 모델을 Vision Task에 적용한 모델로, 전통적인 Convolutional Neural Networks (CNNs) 없이 이미지를 처리합니다. ViT는 이미지를 고정 크기의 패치로 나눈 후, 해당 패치를 선형 임베딩하여 Transformer 모델의 입력으로 사용합니다. 이러한 접근 방식은 자연어 처리에서 성공적으로 사용된 Transformer 구조를 Vision Task에 적용한 것입니다.

### ViT Architecture
- Patch Embedding:
    - 이미지를 고정 크기의 패치로 분할하고, 각 패치를 선형 임베딩하여 고정된 크기의 벡터로 변환합니다.
    - 예를 들어, 224x224 크기의 이미지를 16x16 크기의 패치로 나누면 총 196개의 패치가 생성됩니다.
    - 각 패치는 일련의 픽셀 값을 가지고 있으며, 이 값을 선형 변환하여 고정된 크기의 (임베딩) 벡터로 만듭니다.
- Position Embedding:
    - Transformer는 순차 정보를 처리할 수 없기 때문에 각 패치에 위치 정보를 추가합니다.
    - Position Embedding은 각 패치의 위치를 나타내는 추가적인 벡터를 의미합니다.
- Transformer Encoder:
    - ViT의 핵심 구성 요소입니다.
    - 각 Encoder Layer는 Multi-Head Self Attention(MSA) Module과 Multi Layer Perceptron(MLP) Module로 구성됩니다.
    - Self Attention은 입력 시퀀스 내에서 각 패치가 다른 모든 패치와의 상관관계를 계산하도록 합니다.
    - MLP는 각 패치에 독립적으로 적용되며 비선형 변환을 수행합니다.
    - Layer Normalization(LN)과 Residual Connection이 각 레이어마다 적용됩니다.
- Class Token:
    - 입력 시퀀스의 맨 앞에 학습 가능한 Class Token을 추가합니다.
    - 이 Token은 Transformer Encoder를 거쳐 최종적으로 이미지의 전체적인 표현을 나타냅니다.
- MLP Head:
    - Transformer Encoder의 출력에서 Class Token을 추출한 후, 이를 통해 최종 분류를 수행합니다.
    - 최종 MLP Head는 하나 이상의 선형 층과 활성화 함수로 구성됩니다.

### PyTorch Implementation
```python
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchinfo import summary

# Patch Embedding 모듈: 입력 이미지를 패치로 나누고 임베딩 벡터로 변환하는 모듈
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768, method='conv'):
        super(PatchEmbedding, self).__init__()
        self.img_size = img_size  # 입력 이미지 크기
        self.patch_size = patch_size  # 패치 크기
        self.embed_dim = embed_dim  # 임베딩 벡터의 크기
        self.method = method  # 임베딩 방법 선택: 'conv' 또는 'linear'

        # 이미지 크기를 패치 크기로 나누어 패치의 총 개수 계산
        self.num_patches = (img_size // patch_size) ** 2

        if method == 'conv':
            # 방법 1: Convolutional Layer를 사용하여 패치 임베딩
            self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
        else:
            # 방법 2: Linear Projection을 사용하여 패치 임베딩
            self.proj = nn.Linear(in_channels * (patch_size ** 2), embed_dim)

        # 클래스 토큰 정의: Transformer의 입력에 클래스 토큰을 추가하기 위해 사용
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))

        # 위치 임베딩 정의: 패치와 클래스 토큰의 위치 정보를 추가하기 위해 사용
        self.pos_embed = nn.Parameter(torch.zeros(1, 1 + self.num_patches, embed_dim))

    def forward(self, x):
        # 입력 x의 형태: (batch_size, in_channels, img_size, img_size)

        if self.method == 'conv':
            # 방법 1: Convolutional Layer로 패치 임베딩 수행
            x = self.proj(x)  # x 형태: (batch_size, embed_dim, img_size // patch_size, img_size // patch_size)
            x = x.flatten(2)  # x 형태: (batch_size, embed_dim, num_patches)
            x = x.transpose(1, 2)  # x 형태: (batch_size, num_patches, embed_dim)
        else:
            # 방법 2: Linear Projection으로 패치 임베딩 수행
            batch_size = x.shape[0]  # batch_size
            x = F.unfold(x, kernel_size=self.patch_size, stride=self.patch_size)  # x의 형태: (batch_size, in_channels * patch_size * patch_size, num_patches)
            x = x.transpose(1, 2)  # x의 형태: (batch_size, num_patches, in_channels * patch_size * patch_size)
            x = self.proj(x)  # x의 형태: (batch_size, num_patches, embed_dim)

        # 클래스 토큰을 배치 크기에 맞게 확장
        cls_tokens = self.cls_token.expand(x.shape[0], -1, -1)  # cls_tokens 형태: (batch_size, 1, embed_dim)
        # 클래스 토큰과 패치 임베딩을 결합하여 Transformer의 입력으로 사용
        x = torch.cat((cls_tokens, x), dim=1)  # x 형태: (batch_size, 1 + num_patches, embed_dim)

        # 위치 임베딩을 추가하여 위치 정보를 반영
        x = x + self.pos_embed

        return x


# Multi-Head Self-Attention 모듈: 입력의 각 위치 간의 관계를 학습하는 모듈
# 방법 1: Multi-Head Self-Attention 모듈 직접 구현
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim  # 임베딩 벡터의 크기
        self.num_heads = num_heads  # 어텐션 헤드의 수
        self.head_dim = embed_dim // num_heads  # 각 헤드의 임베딩 크기
        
        # 임베딩 크기가 헤드 수로 정확히 나누어 떨어지는지 확인
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
        
        # 한 번에 Query, Key, Value를 모두 계산
        self.qkv = nn.Linear(embed_dim, embed_dim * 3)
        
        # 최종 출력에 대한 선형 변환 정의
        self.fc_out = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        # 입력 x의 형태: (batch_size, seq_length, embed_dim)
        batch_size, seq_length, embed_dim = x.shape
        
        # Query, Key, Value를 한 번에 계산한 후, 3개로 분리
        qkv = self.qkv(x)  # (batch_size, seq_length, embed_dim * 3)
        Q, K, V = qkv.chunk(3, dim=-1)  # Query, Key, Value로 분리
        
        # Q, K, V를 여러 어텐션 헤드로 나누기 위해 변환
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Scaled Dot-Product Attention 계산
        scores = torch.matmul(Q, K.transpose(-1, -2)) / self.head_dim**0.5  # scores 형태: (batch_size, num_heads, seq_length, seq_length)
        attention = torch.softmax(scores, dim=-1)  # 어텐션 가중치 계산
        out = torch.matmul(attention, V)  # 어텐션 가중치를 통해 Value 계산
        
        # 여러 헤드의 출력을 다시 결합
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_length, embed_dim)
        
        # 최종 선형 변환 적용
        out = self.fc_out(out)
        
        return out
    
# 방법 2: nn.MultiheadAttention 모듈을 사용하여 Multi-Head Self-Attention 구현
""" class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)  # batch_first=True: 입력 텐서의 첫 번째 차원이 batch_size

    def forward(self, x):
        # Self-Attention에서는 Query, Key, Value가 모두 동일
        attn_output, _ = self.attn(x, x, x)
        
        return attn_output """


# Transformer Encoder Block: Multi-Head Self-Attention 모듈과 Multi Layer Perceptron 모듈로 구성된 블록
class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim, dropout=0.1):
        super(TransformerEncoderBlock, self).__init__()
        # 첫 번째 Layer Normalization
        self.norm1 = nn.LayerNorm(embed_dim)

        # Multi-Head Self-Attention 모듈 정의
        self.attention = MultiHeadSelfAttention(embed_dim, num_heads)
        
        # 두 번째 Layer Normalization
        self.norm2 = nn.LayerNorm(embed_dim)

        # Multi Layer Perceptron 모듈 정의
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),  # 임베딩 크기 확장
            nn.GELU(),  # 비선형 활성화 함수
            nn.Linear(ff_hidden_dim, embed_dim),  # 원래 임베딩 크기로 축소
        )
        
        # Dropout 레이어 정의
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        attn_out = self.attention(self.norm1(x))  # Layer Normalization 적용 후 Self-Attention 적용
        x = x + self.dropout(attn_out)  # Residual Connection 적용
        
        mlp_out = self.mlp(self.norm2(x))  # Layer Normalization 적용 후 Multi Layer Perceptron 적용
        x = x + self.dropout(mlp_out)  # Residual Connection 적용
        
        return x


# Transformer Encoder: 여러 개의 Transformer Encoder Block으로 구성된 모듈
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim, num_layers, dropout=0.1):
        super(TransformerEncoder, self).__init__()
        # 여러 개의 TransformerEncoderBlock을 포함하는 리스트 정의
        self.layers = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, ff_hidden_dim, dropout) for _ in range(num_layers)
        ])
    
    def forward(self, x):
        # 모든 Transformer Encoder Block을 순차적으로 통과
        for layer in self.layers:
            x = layer(x)
            
        return x


# Vision Transformer (ViT) 모델 정의
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=1000, 
                 embed_dim=768, depth=12, num_heads=12, ff_hidden_dim=3072, dropout=0.1, method='conv'):
        super(VisionTransformer, self).__init__()
        # Patch Embedding 모듈 정의
        self.patch_embedding = PatchEmbedding(img_size, patch_size, in_channels, embed_dim, method)
        
        # Transformer Encoder 정의
        self.transformer = TransformerEncoder(embed_dim, num_heads, ff_hidden_dim, depth, dropout)
        
        # 최종 출력을 위한 Layer Normalization
        self.norm = nn.LayerNorm(embed_dim)
        
        # 최종 분류를 위한 선형 계층 (MLP Head)
        self.fc = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
        # 입력 이미지를 Patch Embedding을 통해 패치 임베딩으로 변환
        x = self.patch_embedding(x)  # x의 형태: (batch_size, 1 + num_patches, embed_dim)
        
        # Transformer Encoder를 통과하며 피처 추출
        x = self.transformer(x)  # x의 형태: (batch_size, 1 + num_patches, embed_dim)
        x = self.norm(x)  # 마지막 Layer Normalization
        
        # 클래스 토큰 (첫 번째 토큰)을 추출하여 최종 분류 수행
        cls_token = x[:, 0]  # 클래스 토큰 (첫 번째 토큰)
        x = self.fc(cls_token)  # 최종 선형 계층을 통해 클래스 예측
        
        return x


# 모델 사용 예시
if __name__ == "__main__":
    model = VisionTransformer(img_size=224, patch_size=16, num_classes=1000, method='conv')
    input_tensor = torch.randn(16, 3, 224, 224)  # 입력 크기: (16, 3, 224, 224)

    # 모델 구조 요약
    summary(model, input_size=input_tensor.shape, col_width=20, depth=5, row_settings=["depth", "var_names"], col_names=["input_size", "kernel_size", "output_size", "params_percent"])
```